In [1]:
import numpy as np
import pandas as pd
import os
import requests
import io
from datetime import datetime, timedelta
import multiprocessing
import arviz as az
import logging
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
from scipy.stats import poisson
import sqlite3

In [2]:
conn = sqlite3.connect(r"C:\Users\Owner\dev\team-eval\match_db.db")
df = pd.read_sql_query("""SELECT * FROM btb_matches""", conn)

df

,division_id,division,season_id,season,match_date,match_teams,home_team_id,home_team,away_team_id,away_team,...,home_deep,away_deep,home_ppda,away_ppda,bet365_home_odds,bet365_draw_odds,bet365_away_odds,bet365_u25_odds,bet365_o25_odds,match_id
0,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-16 00:00:00,Man United - Fulham,f2b82cdbdadf9d3ec47c3a6be66dcfad,Man United,8cd5e94668b139c1f42a89a1e130f3cf,Fulham,...,7.0,3.0,7.379310,10.833333,1.60,4.20,5.25,2.50,1.53,Man United - Fulham_20240816
1,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-17 00:00:00,Everton - Brighton,6414a61d98ab23b6d757e888ab17a66a,Everton,0d84883ca72c88cb53c8a38262efdcbc,Brighton,...,4.0,5.0,18.333333,7.916667,2.63,3.30,2.63,2.00,1.80,Everton - Brighton_20240817
2,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-17 00:00:00,Ipswich - Liverpool,e4f63bf6d6d2cd121e6c8e59bef68209,Ipswich,afce84ff226407a47c9782a742ba02f7,Liverpool,...,2.0,13.0,18.777778,8.739130,8.50,5.50,1.33,3.00,1.40,Ipswich - Liverpool_20240817
3,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-17 00:00:00,Nott'm Forest - Bournemouth,9a8e1e9fad8766fc3d69a0c26d98b928,Nott'm Forest,b436d55f36cfbe8a085c8b75fb7fe98a,Bournemouth,...,10.0,4.0,8.653846,9.954545,2.45,3.50,2.80,2.10,1.73,Nott'm Forest - Bournemouth_20240817
4,98b8784f6685b7289f583e0ce4b4f6f2,Premier League,3ac445d3cc1d404987efdfcfa42f3bcd,20242025,2024-08-17 00:00:00,Newcastle - Southampton,78e9266876e7649e0a12e3840f5be006,Newcastle,5a884401673693b0bdf379fefb7ec2b2,Southampton,...,4.0,13.0,16.250000,3.789474,1.36,5.25,8.00,3.00,1.40,Newcastle - Southampton_20240817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3536,c4781225ef2d5018f7a9df4b6cb1c1fe,Championship,06b84148ed1c6a06340478bef065f5bc,20212022,2022-05-07 00:00:00,Huddersfield - Bristol City,5668c64f7d3e02ee708188dbe0fdaa4e,Huddersfield,b69f5235edbfe00c680f756eba63f7be,Bristol City,...,NaN,NaN,NaN,NaN,1.90,3.60,3.90,2.10,1.72,Huddersfield - Bristol City_20220507
3537,c4781225ef2d5018f7a9df4b6cb1c1fe,Championship,06b84148ed1c6a06340478bef065f5bc,20212022,2022-05-07 00:00:00,Luton - Reading,20ee5d666fa2428304848c9ebf2cf8b2,Luton,26fccddf2f94b1685b184267996e10f8,Reading,...,NaN,NaN,NaN,NaN,1.50,4.20,7.00,2.20,1.66,Luton - Reading_20220507
3538,c4781225ef2d5018f7a9df4b6cb1c1fe,Championship,06b84148ed1c6a06340478bef065f5bc,20212022,2022-05-07 00:00:00,Sheffield United - Fulham,851eb9d41bc5f8ef6237ecb99c9720ce,Sheffield United,8cd5e94668b139c1f42a89a1e130f3cf,Fulham,...,NaN,NaN,NaN,NaN,2.00,3.50,3.75,2.00,1.80,Sheffield United - Fulham_20220507
3539,c4781225ef2d5018f7a9df4b6cb1c1fe,Championship,06b84148ed1c6a06340478bef065f5bc,20212022,2022-05-07 00:00:00,Bournemouth - Millwall,b436d55f36cfbe8a085c8b75fb7fe98a,Bournemouth,ea11a8d754f9bbf49dc01d6d3ae875d9,Millwall,...,NaN,NaN,NaN,NaN,1.95,3.60,3.80,2.10,1.72,Bournemouth - Millwall_20220507


In [4]:
final_df = pd.read_sql_query("""SELECT * FROM xgb_match_features_0065_001""", conn)

final_df

,division,season,match_date,match_id,team,avg_market_value,goals,shots,shots_on_target,corners,...,opponent_rolling_shots_on_target_for_30d,opponent_rolling_shots_on_target_against_30d,opponent_rolling_corners_for_30d,opponent_rolling_corners_against_30d,opponent_rolling_deep_for_30d,opponent_rolling_deep_against_30d,opponent_rolling_ppda_for_30d,opponent_rolling_ppda_against_30d,opponent_rolling_odds_for_30d,opponent_rolling_odds_against_30d
0,Premier League,20212022,2021-08-13 00:00:00,Brentford - Arsenal_20210813,Arsenal,19750000,0,22.0,4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Premier League,20212022,2021-08-22 00:00:00,Arsenal - Chelsea_20210822,Arsenal,19888889,0,6.0,3,9,...,6.000000,1.000000,5.000000,2.000000,14.000000,3.000000,6.600000,71.000000,1.250000,13.000000
2,Premier League,20212022,2021-08-28 00:00:00,Man City - Arsenal_20210828,Arsenal,19888889,0,1.0,0,0,...,4.000000,1.495500,8.492500,1.997000,12.012000,2.492500,6.882220,22.450435,1.339220,15.780750
3,Premier League,20212022,2021-09-11 00:00:00,Arsenal - Norwich_20210911,Arsenal,21096154,1,30.0,7,8,...,2.335686,4.988342,4.347360,8.662022,1.664347,10.643290,21.368960,9.316791,13.156062,1.411286
4,Premier League,20212022,2021-09-18 00:00:00,Burnley - Arsenal_20210918,Arsenal,21096154,1,13.0,3,3,...,3.762811,6.237039,5.499293,5.737952,3.003786,7.749459,13.948550,8.768627,6.043155,1.893619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7077,Premier League,20242025,2025-02-16 00:00:00,Liverpool - Wolves_20250216,Wolves,10972727,1,16.0,4,3,...,6.785002,3.693686,7.156575,2.239583,15.678170,4.090837,7.230244,14.780322,1.446936,7.793515
7078,Premier League,20242025,2025-02-22 00:00:00,Bournemouth - Wolves_20250222,Wolves,10972727,1,13.0,5,7,...,7.485164,4.998216,4.502694,5.733792,5.481596,10.479800,7.312345,8.240788,3.252346,3.187490
7079,Premier League,20242025,2025-02-25 00:00:00,Wolves - Fulham_20250225,Wolves,10972727,1,18.0,5,7,...,4.249590,3.010004,6.264898,3.773559,5.518628,5.989220,15.824290,18.219634,2.819511,2.954952
7080,Premier League,20242025,2025-03-08 00:00:00,Wolves - Everton_20250308,Wolves,10972727,1,11.0,3,5,...,6.018758,3.308347,4.562368,5.553719,6.620515,5.726807,18.604640,10.016210,3.491567,2.698088


In [8]:
final_df["match_date"] = pd.to_datetime(final_df["match_date"])

final_df = final_df[final_df["match_date"] > '2021-12-31']

final_df = final_df[final_df["division"] == "Premier League"]
final_df["xg_interaction"] = final_df["rolling_xg_for"] * final_df["opponent_rolling_xg_against"]
final_df["rolling_finishing_ability"] = final_df["rolling_goals_for"] - final_df["rolling_xg_for"]
final_df["rolling_opponent_finishing_ability"] = final_df["opponent_rolling_goals_against"] - final_df["opponent_rolling_xg_against"]

feature_cols = ["division", "home?", "avg_market_value", "opponent_avg_market_value", "rolling_finishing_ability", "rolling_opponent_finishing_ability", #"team_season", "opponent_team_season",#"xg_interaction",
             "rolling_goals_for", "rolling_xg_for", "rolling_shots_for", "rolling_shots_on_target_for", "rolling_corners_for", "rolling_deep_for", "rolling_ppda_for", 'rolling_goals_for_30d', 'rolling_goals_against_30d', 'rolling_xg_for_30d', 'rolling_xg_against_30d', 'rolling_shots_for_30d', 'rolling_shots_against_30d', 'rolling_shots_on_target_for_30d', 'rolling_shots_on_target_against_30d', 'rolling_corners_for_30d', 'rolling_corners_against_30d', 'rolling_deep_for_30d', 'rolling_deep_against_30d', 'rolling_ppda_for_30d', 'rolling_ppda_against_30d', 
             "opponent_rolling_goals_against", "opponent_rolling_xg_against", "opponent_rolling_shots_against", "opponent_rolling_shots_on_target_against", "opponent_rolling_corners_against", "opponent_rolling_deep_against", "opponent_rolling_ppda_against", 'opponent_rolling_goals_for_30d', 'opponent_rolling_goals_against_30d', 'opponent_rolling_xg_for_30d', 'opponent_rolling_xg_against_30d', 'opponent_rolling_shots_for_30d', 'opponent_rolling_shots_against_30d', 'opponent_rolling_shots_on_target_for_30d', 'opponent_rolling_shots_on_target_against_30d', 'opponent_rolling_corners_for_30d', 'opponent_rolling_corners_against_30d', 'opponent_rolling_deep_for_30d', 'opponent_rolling_deep_against_30d', 'opponent_rolling_ppda_for_30d', 'opponent_rolling_ppda_against_30d']

X = final_df[feature_cols]

# Convert categorical columns to category type
cat_cols = ["division"] #"team", "opponent_team"
for col in cat_cols:
    X[col] = X[col].astype('category')


y = final_df["goals"] 

text_cutoff = pd.to_datetime('2024-08-01')
X_train = X[final_df['match_date'] < text_cutoff]
y_train = y[final_df['match_date'] < text_cutoff]
X_test = X[final_df['match_date'] >= text_cutoff]
y_test = y[final_df['match_date'] >= text_cutoff]

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")

model_params = {
        'n_estimators': 100,
        'learning_rate': 0.03,
        'max_depth': 5,
        'subsample': 0.7,
        'colsample_bytree': 0.9,
        'min_child_weight': 3,
        'gamma': 0.1,
        'alpha': 1,
        #'lambda_': 1,
        'random_state': 26,
        #'enable_categorical': True
    }

# Train an XGBoost model
model = xgb.XGBRegressor(objective='count:poisson', **model_params, enable_categorical= True)

model.fit(X_train, y_train)

# Make predictions
y_train_pred = model.predict(X_train)
y_pred = model.predict(X_test)

# Evaluate model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
print(f"RMSE: {rmse:.4f}")
print(f"Test MAE: {mae:.4f}")
print(f"Train MAE: {train_mae:.4f}")

X_test_has_nan = X_test.isna().any(axis=1)
y_test_has_nan = y_test.isna()

print(f"X_test rows with NaN: {X_test_has_nan.sum()}")
print(f"y_test rows with NaN: {y_test_has_nan.sum()}")

C:\Users\Owner\AppData\Local\Temp\ipykernel_22324\920584210.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype('category')


Training set size: 1914 samples
Test set size: 578 samples
RMSE: 1.1750
Test MAE: 0.9259
Train MAE: 0.8220
X_test rows with NaN: 2
y_test rows with NaN: 0


In [9]:
# Get the test set indexes
test_indices = final_df.index[final_df['match_date'] >= text_cutoff]

# Create a temporary dataframe with all the test data
temp_df = final_df.loc[test_indices].copy()

# Initialize the predictions dataframe
match_preds = []

# Group by match_id to combine home and away rows
for match_id, group in temp_df.groupby('match_id'):
    if len(group) != 2:
        print(f"Warning: Match ID {match_id} does not have exactly 2 rows (has {len(group)})")
        continue
        
    # Get home and away rows
    home_row = group[group['home?'] == 1]
    away_row = group[group['home?'] == 0]
    
    if len(home_row) != 1 or len(away_row) != 1:
        print(f"Warning: Match ID {match_id} doesn't have one home and one away team")
        continue
    
    # Get the model predictions for both teams
    home_idx = home_row.index[0]
    away_idx = away_row.index[0]
    
    home_pred = y_pred[np.where(test_indices == home_idx)[0][0]] if home_idx in test_indices else None
    away_pred = y_pred[np.where(test_indices == away_idx)[0][0]] if away_idx in test_indices else None
    
    # Create a single row for this match
    match_data = {
        'match_id': match_id,
        'match_date': home_row['match_date'].values[0],
        'home_team': home_row['team'].values[0],
        'away_team': away_row['team'].values[0],
        'division': home_row['division'].values[0],
        'actual_home_goals': home_row['goals'].values[0],
        'actual_away_goals': away_row['goals'].values[0],
        'predicted_home_goals': home_pred,
        'predicted_away_goals': away_pred,
        'home_error': home_row['goals'].values[0] - home_pred if home_pred is not None else None,
        'away_error': away_row['goals'].values[0] - away_pred if away_pred is not None else None
    }
    
    # Add match_teams if available
    if 'match_teams' in home_row.columns:
        match_data['match_teams'] = home_row['match_teams'].values[0]
    
    match_preds.append(match_data)

# Create the final predictions dataframe
preds_df = pd.DataFrame(match_preds)

# Add some calculated columns
preds_df['home_abs_error'] = np.abs(preds_df['home_error'])
preds_df['away_abs_error'] = np.abs(preds_df['away_error'])
preds_df['total_abs_error'] = preds_df['home_abs_error'] + preds_df['away_abs_error']

# Save to CSV
#preds_df.to_csv("match_predictions.csv", index=False)

# Display summary stats
print("\nPrediction Summary:")
print(f"Total matches predicted: {len(preds_df)}")
print(f"Home goals MAE: {preds_df['home_abs_error'].mean():.4f}")
print(f"Away goals MAE: {preds_df['away_abs_error'].mean():.4f}")
print(f"Overall MAE: {(preds_df['home_abs_error'].mean() + preds_df['away_abs_error'].mean())/2:.4f}")
print(f"Date range: {preds_df['match_date'].min().date()} to {preds_df['match_date'].max().date()}")

preds_df


Prediction Summary:
Total matches predicted: 289
Home goals MAE: 0.9620
Away goals MAE: 0.8898
Overall MAE: 0.9259
Date range: 2024-08-16 to 2025-03-16


,match_id,match_date,home_team,away_team,division,actual_home_goals,actual_away_goals,predicted_home_goals,predicted_away_goals,home_error,away_error,home_abs_error,away_abs_error,total_abs_error
0,Arsenal - Aston Villa_20250118,2025-01-18,Arsenal,Aston Villa,Premier League,2,2,2.135503,1.077202,-0.135503,0.922798,0.135503,0.922798,1.058301
1,Arsenal - Brighton_20240831,2024-08-31,Arsenal,Brighton,Premier League,1,1,1.981994,1.222279,-0.981994,-0.222279,0.981994,0.222279,1.204273
2,Arsenal - Chelsea_20250316,2025-03-16,Arsenal,Chelsea,Premier League,1,0,1.946192,1.274072,-0.946192,-1.274072,0.946192,1.274072,2.220264
3,Arsenal - Everton_20241214,2024-12-14,Arsenal,Everton,Premier League,0,0,2.227149,0.888304,-2.227149,-0.888304,2.227149,0.888304,3.115453
4,Arsenal - Ipswich_20241227,2024-12-27,Arsenal,Ipswich,Premier League,1,0,2.622260,0.840744,-1.622260,-0.840744,1.622260,0.840744,2.463003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,Wolves - Man City_20241020,2024-10-20,Wolves,Man City,Premier League,1,2,0.976156,2.145340,0.023844,-0.145340,0.023844,0.145340,0.169184
285,Wolves - Man United_20241226,2024-12-26,Wolves,Man United,Premier League,2,0,1.117664,1.633107,0.882336,-1.633107,0.882336,1.633107,2.515444
286,Wolves - Newcastle_20240915,2024-09-15,Wolves,Newcastle,Premier League,1,2,1.158830,1.952790,-0.158830,0.047210,0.158830,0.047210,0.206039
287,Wolves - Nott'm Forest_20250106,2025-01-06,Wolves,Nott'm Forest,Premier League,0,3,1.114475,1.175367,-1.114475,1.824633,1.114475,1.824633,2.939107


In [10]:
def append_df_to_sqlite_table(df_new, db_path, table_name, key_columns=None, batch_size=500):
    # Make a copy to avoid modifying the original dataframe
    df = df_new.copy()
    
    # Connect to the database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Check if table exists
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
    table_exists = cursor.fetchone() is not None
    
    if table_exists:
        # If key_columns not specified, use all columns
        if key_columns is None:
            key_columns = list(df.columns)
        
        # Get columns from existing table
        cursor.execute(f"PRAGMA table_info({table_name})")
        existing_columns = [row[1] for row in cursor.fetchall()]
        
        # Ensure all key columns exist in both dataframe and table
        key_columns = [col for col in key_columns if col in df.columns and col in existing_columns]
        
        if key_columns:
            # Process in batches to avoid SQL expression tree too large error
            filtered_dfs = []
            
            # Split dataframe into batches
            total_rows = len(df)
            for start_idx in range(0, total_rows, batch_size):
                end_idx = min(start_idx + batch_size, total_rows)
                batch_df = df.iloc[start_idx:end_idx]
                
                # For this batch, extract all key column values
                key_values = {}
                for key in key_columns:
                    key_values[key] = batch_df[key].tolist()
                
                # Build an optimized query - using IN clause for each key column
                where_clauses = []
                params = []
                
                for key in key_columns:
                    if len(key_values[key]) > 0:
                        placeholders = ','.join(['?'] * len(key_values[key]))
                        where_clauses.append(f"{key} IN ({placeholders})")
                        params.extend(key_values[key])
                
                if where_clauses:
                    # Get existing records
                    query = f"SELECT {', '.join(key_columns)} FROM {table_name} WHERE {' OR '.join(where_clauses)}"
                    cursor.execute(query, params)
                    existing_rows = cursor.fetchall()
                    
                    # Create a set of tuples representing existing rows
                    if key_columns:
                        existing_keys = set(existing_rows)
                        
                        # Filter out rows that already exist
                        batch_filtered = batch_df[~batch_df[key_columns].apply(tuple, axis=1).isin(existing_keys)]
                        if not batch_filtered.empty:
                            filtered_dfs.append(batch_filtered)
                else:
                    # No valid key values, include all rows
                    filtered_dfs.append(batch_df)
            
            # Combine all filtered batches
            if filtered_dfs:
                df_filtered = pd.concat(filtered_dfs, ignore_index=True)
                if not df_filtered.empty:
                    df_filtered.to_sql(table_name, conn, if_exists='append', index=False)
                    print(f"Added {len(df_filtered)} new records to table {table_name}")
                else:
                    print("No new records to append.")
            else:
                print("No new records to append.")
        else:
            # If no valid key columns, append all data (may create duplicates)
            df.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"Warning: No valid key columns. Added all {len(df)} records to table {table_name}")
    else:
        # If table doesn't exist, create it with the new data
        df.to_sql(table_name, conn, index=False)
        print(f"Created new table {table_name} with {len(df)} records")
    
    # Close the connection
    conn.close()

append_df_to_sqlite_table(preds_df, 'match_db.db', 'xgb_pred_0065_001_prem20242025', key_columns=['match_id'])

Created new table xgb_pred_0065_001_prem20242025 with 289 records


In [14]:
backtest_df = pd.read_sql_query("""
    SELECT
        preds.match_id,
        preds.predicted_home_goals,
        preds.actual_home_goals,
        preds.predicted_away_goals,
        preds.actual_away_goals,
        btb.bet365_home_odds,
        btb.bet365_draw_odds,
        btb.bet365_away_odds,
        btb.bet365_u25_odds,
        btb.bet365_o25_odds
    FROM
        xgb_pred_0065_001_prem20242025 preds
    JOIN
        btb_matches btb
            ON btb.match_id = preds.match_id
                                
    """, conn)

backtest_df

,match_id,predicted_home_goals,actual_home_goals,predicted_away_goals,actual_away_goals,bet365_home_odds,bet365_draw_odds,bet365_away_odds,bet365_u25_odds,bet365_o25_odds
0,Arsenal - Aston Villa_20250118,2.135503,2,1.077202,2,1.50,4.10,7.00,2.03,1.87
1,Arsenal - Brighton_20240831,1.981994,1,1.222279,1,1.33,5.50,8.50,2.50,1.53
2,Arsenal - Chelsea_20250316,1.946192,1,1.274072,0,1.80,3.60,4.50,1.92,1.86
3,Arsenal - Everton_20241214,2.227149,0,0.888304,0,1.22,6.50,12.00,2.20,1.67
4,Arsenal - Ipswich_20241227,2.622260,1,0.840744,0,1.14,9.00,15.00,3.00,1.40
...,...,...,...,...,...,...,...,...,...,...
284,Wolves - Man City_20241020,0.976156,1,2.145340,2,8.00,6.00,1.33,2.75,1.44
285,Wolves - Man United_20241226,1.117664,2,1.633107,0,3.90,3.70,1.91,2.20,1.67
286,Wolves - Newcastle_20240915,1.158830,1,1.952790,2,3.40,3.50,2.10,2.38,1.57
287,Wolves - Nott'm Forest_20250106,1.114475,0,1.175367,3,3.30,3.20,2.30,1.73,2.10


In [18]:
def simulate_match_outcomes(home_goals, away_goals, num_simulations=10000):

    # Generate random samples from Poisson distributions
    home_scores = np.random.poisson(home_goals, num_simulations)
    away_scores = np.random.poisson(away_goals, num_simulations)
    
    # Calculate total goals for each simulation
    total_goals = home_scores + away_scores
    
    # Count outcomes
    home_wins = np.sum(home_scores > away_scores)
    draws = np.sum(home_scores == away_scores)
    away_wins = np.sum(home_scores < away_scores)
    
    # Count over/under 2.5 goals
    over_2_5 = np.sum(total_goals > 2.5)
    under_2_5 = np.sum(total_goals <= 2.5)
    
    # Convert to probabilities
    home_win_prob = home_wins / num_simulations
    draw_prob = draws / num_simulations
    away_win_prob = away_wins / num_simulations
    over_2_5_prob = over_2_5 / num_simulations
    under_2_5_prob = under_2_5 / num_simulations
    
    return {
        'home_win': home_win_prob,
        'draw': draw_prob,
        'away_win': away_win_prob,
        'over_2_5': over_2_5_prob,
        'under_2_5': under_2_5_prob
    }

# Add these probabilities to your prediction dataframe
backtest_df['probs'] = backtest_df.apply(
    lambda row: simulate_match_outcomes(row['predicted_home_goals'], row['predicted_away_goals']), 
    axis=1
)

# Extract individual probabilities to separate columns
backtest_df['home_win_prob'] = backtest_df['probs'].apply(lambda x: x['home_win'])
backtest_df['draw_prob'] = backtest_df['probs'].apply(lambda x: x['draw'])
backtest_df['away_win_prob'] = backtest_df['probs'].apply(lambda x: x['away_win'])
backtest_df['over_2_5_prob'] = backtest_df['probs'].apply(lambda x: x['over_2_5'])
backtest_df['under_2_5_prob'] = backtest_df['probs'].apply(lambda x: x['under_2_5'])

# Drop the intermediate dictionary column
backtest_df = backtest_df.drop('probs', axis=1)

append_df_to_sqlite_table(backtest_df, 'match_db.db', 'xgb_backtest_0065_001_prem20242025', key_columns=['match_id'])

backtest_df

Created new table xgb_backtest_0065_001_prem20242025 with 289 records


,match_id,predicted_home_goals,actual_home_goals,predicted_away_goals,actual_away_goals,bet365_home_odds,bet365_draw_odds,bet365_away_odds,bet365_u25_odds,bet365_o25_odds,home_win_prob,draw_prob,away_win_prob,over_2_5_prob,under_2_5_prob
0,Arsenal - Aston Villa_20250118,2.135503,2,1.077202,2,1.50,4.10,7.00,2.03,1.87,0.6132,0.2024,0.1844,0.6199,0.3801
1,Arsenal - Brighton_20240831,1.981994,1,1.222279,1,1.33,5.50,8.50,2.50,1.53,0.5531,0.2138,0.2331,0.6264,0.3736
2,Arsenal - Chelsea_20250316,1.946192,1,1.274072,0,1.80,3.60,4.50,1.92,1.86,0.5278,0.2277,0.2445,0.6269,0.3731
3,Arsenal - Everton_20241214,2.227149,0,0.888304,0,1.22,6.50,12.00,2.20,1.67,0.6827,0.1840,0.1333,0.6081,0.3919
4,Arsenal - Ipswich_20241227,2.622260,1,0.840744,0,1.14,9.00,15.00,3.00,1.40,0.7524,0.1524,0.0952,0.6669,0.3331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,Wolves - Man City_20241020,0.976156,1,2.145340,2,8.00,6.00,1.33,2.75,1.44,0.1619,0.1895,0.6486,0.6019,0.3981
285,Wolves - Man United_20241226,1.117664,2,1.633107,0,3.90,3.70,1.91,2.20,1.67,0.2627,0.2455,0.4918,0.5158,0.4842
286,Wolves - Newcastle_20240915,1.158830,1,1.952790,2,3.40,3.50,2.10,2.38,1.57,0.2213,0.2183,0.5604,0.5979,0.4021
287,Wolves - Nott'm Forest_20250106,1.114475,0,1.175367,3,3.30,3.20,2.30,1.73,2.10,0.3472,0.2834,0.3694,0.4073,0.5927


In [36]:
def kelly_expected(probability, decimal_odds, fraction = 1.0):
    if decimal_odds <= 1 or probability <= 0 or probability >= 1:
        return 0, 0
        
    # Calculate Kelly Criterion
    q = 1 - probability  # probability of losing
    kelly = (probability * (decimal_odds - 1) - q) / (decimal_odds - 1) # win rate * potential profit subtracitng prob of losing. Divided by potentail profit
    kelly = kelly * fraction  # Apply fractional Kelly
    kelly = max(0, kelly)  # No negative bets
    
    # Calculate Expected Value
    ev = (probability * (decimal_odds - 1)) - (1 - probability)
    ev = ev  # Convert to percentage
    
    return kelly, ev

prob = 0.6398
odds = 1.67
fraction = 0.25

bet_size, ev = kelly_expected(prob, odds, fraction)
print(f"Optimal bet size: {bet_size:.1%}")
print(f"Expected Value: {ev:.1%}")


Optimal bet size: 2.6%
Expected Value: 6.8%


In [62]:
home_win_df = pd.read_sql_query("""SELECT match_id, actual_home_goals, actual_away_goals, home_win_prob, bet365_home_odds FROM xgb_backtest_0065_001_prem20242025""", conn)

home_win_df["home_win?"] = (home_win_df["actual_home_goals"] > home_win_df["actual_away_goals"]).astype(int)

results = home_win_df.apply(
    lambda row: kelly_expected(row['home_win_prob'], row['bet365_home_odds'], fraction=0.25),
    axis=1
)

# Extract Kelly and EV values into separate columns
home_win_df['kelly_stake'] = results.apply(lambda x: x[0])
home_win_df['ev_percent'] = results.apply(lambda x: x[1])
home_win_df['is_value_bet'] = home_win_df['ev_percent'] > 0.02
#print(home_win_df)

home_win_df['profit'] = home_win_df.apply(
    lambda row: row['kelly_stake'] * (row['bet365_home_odds'] - 1) if row['is_value_bet'] and row['home_win?'] == 1 
              else (-row['kelly_stake'] if row['is_value_bet'] else 0),  # Lose Kelly stake for lost bets
    axis=1
)

# Display summary statistics
value_bets = home_win_df[home_win_df['is_value_bet']]
print(f"Total matches analyzed: {len(home_win_df)}")
print(f"Value bets found: {len(value_bets)} ({len(value_bets)/len(home_win_df):.1%} of total)")

if len(value_bets) > 0:
    # Calculate performance of value bets
    won_bets = value_bets['home_win?'].sum()
    profit = value_bets['profit'].sum()
    
    print(f"Value bets won: {won_bets}/{len(value_bets)} ({won_bets/len(value_bets):.1%})")
    print(f"Total profit: {profit:.2f} units")
    print(f"ROI: {profit/len(value_bets)*100:.2f}%")

Total matches analyzed: 289
Value bets found: 68 (23.5% of total)
Value bets won: 13/68 (19.1%)
Total profit: -0.55 units
ROI: -0.81%


In [61]:
away_win_df = pd.read_sql_query("""SELECT match_id, actual_home_goals, actual_away_goals, away_win_prob, bet365_away_odds FROM xgb_backtest_0065_001_prem20242025""", conn)

away_win_df["away_win?"] = (away_win_df["actual_away_goals"] > away_win_df["actual_home_goals"]).astype(int)

results = away_win_df.apply(
    lambda row: kelly_expected(row['away_win_prob'], row['bet365_away_odds'], fraction=0.25),
    axis=1
)

# Extract Kelly and EV values into separate columns
away_win_df['kelly_stake'] = results.apply(lambda x: x[0])
away_win_df['ev_percent'] = results.apply(lambda x: x[1])
away_win_df['is_value_bet'] = away_win_df['ev_percent'] > 0.02

away_win_df['profit'] = away_win_df.apply(
    lambda row: row['kelly_stake'] * (row['bet365_away_odds'] - 1) if row['is_value_bet'] and row['away_win?'] == 1 
              else (-row['kelly_stake'] if row['is_value_bet'] else 0),  # Lose Kelly stake for lost bets
    axis=1
)

# Display summary statistics
value_bets = away_win_df[away_win_df['is_value_bet']]
print(f"Total matches analyzed: {len(away_win_df)}")
print(f"Value bets found: {len(value_bets)} ({len(value_bets)/len(away_win_df):.1%} of total)")

if len(value_bets) > 0:
    # Calculate performance of value bets
    won_bets = value_bets['away_win?'].sum()
    profit = value_bets['profit'].sum()
    
    print(f"Value bets won: {won_bets}/{len(value_bets)} ({won_bets/len(value_bets):.1%})")
    print(f"Total profit: {profit:.2f} units")
    print(f"ROI: {profit/len(value_bets)*100:.2f}%")

Total matches analyzed: 289
Value bets found: 185 (64.0% of total)
Value bets won: 51/185 (27.6%)
Total profit: 0.20 units
ROI: 0.11%


In [60]:
draw_df = pd.read_sql_query("""SELECT match_id, actual_home_goals, actual_away_goals, draw_prob, bet365_draw_odds FROM xgb_backtest_0065_001_prem20242025""", conn)

draw_df["draw?"] = (draw_df["actual_away_goals"] == draw_df["actual_home_goals"]).astype(int)

results = draw_df.apply(
    lambda row: kelly_expected(row['draw_prob'], row['bet365_draw_odds'], fraction=0.25),
    axis=1
)

# Extract Kelly and EV values into separate columns
draw_df['kelly_stake'] = results.apply(lambda x: x[0])
draw_df['ev_percent'] = results.apply(lambda x: x[1])
draw_df['is_value_bet'] = draw_df['ev_percent'] > 0.02

draw_df['profit'] = draw_df.apply(
    lambda row: row['kelly_stake'] * (row['bet365_draw_odds'] - 1) if row['is_value_bet'] and row['draw?'] == 1 
              else (-row['kelly_stake'] if row['is_value_bet'] else 0),  # Lose Kelly stake for lost bets
    axis=1
)

# Display summary statistics
value_bets = draw_df[draw_df['is_value_bet']]
print(f"Total matches analyzed: {len(draw_df)}")
print(f"Value bets found: {len(value_bets)} ({len(value_bets)/len(draw_df):.1%} of total)")

if len(value_bets) > 0:
    # Calculate performance of value bets
    won_bets = value_bets['draw?'].sum()
    profit = value_bets['profit'].sum()
    
    print(f"Value bets won: {won_bets}/{len(value_bets)} ({won_bets/len(value_bets):.1%})")
    print(f"Total profit: {profit:.2f} units")
    print(f"ROI: {profit/len(value_bets)*100:.2f}%")

Total matches analyzed: 289
Value bets found: 67 (23.2% of total)
Value bets won: 16/67 (23.9%)
Total profit: -0.03 units
ROI: -0.05%


In [70]:
over_25_df = pd.read_sql_query("""SELECT match_id, actual_home_goals, actual_away_goals, over_2_5_prob, bet365_o25_odds FROM xgb_backtest_0065_001_prem20242025""", conn)

over_25_df["over_25?"] = (over_25_df["actual_away_goals"] + over_25_df["actual_home_goals"] > 2.5).astype(int)

results = over_25_df.apply(
    lambda row: kelly_expected(row['over_2_5_prob'], row['bet365_o25_odds'], fraction=0.25),
    axis=1
)

# Extract Kelly and EV values into separate columns
over_25_df['kelly_stake'] = results.apply(lambda x: x[0])
over_25_df['ev_percent'] = results.apply(lambda x: x[1])
over_25_df['is_value_bet'] = over_25_df['ev_percent'] > 0.02

over_25_df['profit'] = over_25_df.apply(
    lambda row: row['kelly_stake'] * (row['bet365_o25_odds'] - 1) if row['is_value_bet'] and row['over_25?'] == 1 
              else (-row['kelly_stake'] if row['is_value_bet'] else 0),  # Lose Kelly stake for lost bets
    axis=1
)

# Display summary statistics
value_bets = over_25_df[over_25_df['is_value_bet']]
print(f"Total matches analyzed: {len(over_25_df)}")
print(f"Value bets found: {len(value_bets)} ({len(value_bets)/len(over_25_df):.1%} of total)")

if len(value_bets) > 0:
    # Calculate performance of value bets
    won_bets = value_bets['over_25?'].sum()
    profit = value_bets['profit'].sum()
    
    print(f"Value bets won: {won_bets}/{len(value_bets)} ({won_bets/len(value_bets):.1%})")
    print(f"Total profit: {profit:.2f} units")
    print(f"ROI: {profit/len(value_bets)*100:.2f}%")

Total matches analyzed: 289
Value bets found: 64 (22.1% of total)
Value bets won: 33/64 (51.6%)
Total profit: -0.05 units
ROI: -0.08%


In [69]:
under_25_df = pd.read_sql_query("""SELECT match_id, actual_home_goals, actual_away_goals, under_2_5_prob, bet365_u25_odds FROM xgb_backtest_0065_001_prem20242025""", conn)

under_25_df["under_25?"] = (under_25_df["actual_away_goals"] + under_25_df["actual_home_goals"] <= 2.5).astype(int)

results = under_25_df.apply(
    lambda row: kelly_expected(row['under_2_5_prob'], row['bet365_u25_odds'], fraction=0.25),
    axis=1
)

# Extract Kelly and EV values into separate columns
under_25_df['kelly_stake'] = results.apply(lambda x: x[0])
under_25_df['ev_percent'] = results.apply(lambda x: x[1])
under_25_df['is_value_bet'] = under_25_df['ev_percent'] > 0.02

under_25_df['profit'] = under_25_df.apply(
    lambda row: row['kelly_stake'] * (row['bet365_u25_odds'] - 1) if row['is_value_bet'] and row['under_25?'] == 1 
              else (-row['kelly_stake'] if row['is_value_bet'] else 0),  # Lose Kelly stake for lost bets
    axis=1
)

# Display summary statistics
value_bets = under_25_df[under_25_df['is_value_bet']]
print(f"Total matches analyzed: {len(under_25_df)}")
print(f"Value bets found: {len(value_bets)} ({len(value_bets)/len(under_25_df):.1%} of total)")

if len(value_bets) > 0:
    # Calculate performance of value bets
    won_bets = value_bets['under_25?'].sum()
    profit = value_bets['profit'].sum()
    
    print(f"Value bets won: {won_bets}/{len(value_bets)} ({won_bets/len(value_bets):.1%})")
    print(f"Total profit: {profit:.2f} units")
    print(f"ROI: {profit/len(value_bets)*100:.2f}%")

Total matches analyzed: 289
Value bets found: 98 (33.9% of total)
Value bets won: 36/98 (36.7%)
Total profit: 0.17 units
ROI: 0.18%
